In [11]:
from src.vectorstore import connect_to_milvus, activate_collection
from src.retrievers import MilvusRerankRetriever

import os
from dotenv import load_dotenv

In [12]:
from src.retrievers import MilvusMultiVectorRetriever

from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

import os
from dotenv import load_dotenv


In [13]:
connect_to_milvus()
collection = activate_collection()
collection.load()

In [14]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

In [15]:
# Template

"""If you do not have the answer in the CONTEXT write "I don't know".
Do not make things up. Do not fabricate facts. """
RAG_TEMPLATE = """\
You are a Huberman Lab podcast listener. 
You will be provided a CONTEXT of the podcasts and a QUESTION.
Based on the CONTEXT answer the QUESTION.

Context:
{context}

Query:
{question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

# Setup a chat model
chat_model = ChatOpenAI()
# Setup a retriever
retriever = MilvusMultiVectorRetriever()

# Setup a chain
multivector_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [16]:
res = multivector_chain.invoke({"question" : "How much caffeine a day is the maximum limit?"})

In [17]:
res

{'response': AIMessage(content='The maximum limit of caffeine recommended per day is 600 milligrams, as mentioned in the podcast.', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 2922, 'total_tokens': 2942}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-7d691e15-8822-421d-9b28-680331dac152-0'),
 'context': [[Document(page_content="We'll talk about the use of caffeine every other day.\nI know for you habitual caffeine drinkers, including myself, just the simple mention of that probably sounds aversive, but there is actually great utility to using caffeine every other day as opposed to every day.\nBut just keep in mind that some people will drink caffeine and not get much of a lift from it at all.\nOther people will drink caffeine and they will feel extremely anxious, even at dosages far lower than that one to three milligrams per kilogram of body weight range that I described a mome